In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import openpyxl
import xlrd
from rdkit import Chem
from rdkit.Chem import inchi
from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardizer as Papyrus_standardizer
from papyrus_structure_pipeline import standardize

from papyrus_scripts import *

[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>, ForwardRef('Index'), ForwardRef('Series')]
[<class 'str'>, <class 'float'>, <class 'bool'>]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta')]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval')]
[<class 'str'>, <class 'float'>, <class 'bool'>, ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval'), <class 'numpy.datetime64'>, <class 'numpy.timedelta64'>, <class 'datetime.datetime'>]
[ForwardRef('Timestamp'), <class 'datetime.datetime'>, <class 'numpy.datetime64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[ForwardRef('Timedelta'), <class 'datetime.timedelta'>, <class 'numpy.timedelta64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[<class 'str'>, <class 'datetime.tzinfo'>]
[<class 'str'>, <class 'int'>]
[typing.Hashable, typing.Sequence[typing.Ha

[19:16:03] Initializing Normalizer


[<class 'rdkit.Chem.rdchem.Mol'>, <class 'NoneType'>]
[<class 'rdkit.Chem.rdchem.Mol'>, typing.Tuple[typing.Optional[rdkit.Chem.rdchem.Mol], papyrus_structure_pipeline.standardizer.StandardizationResult]]
[<class 'bool'>, typing.Tuple[bool, papyrus_structure_pipeline.standardizer.InorganicSubtype]]
[<class 'rdkit.Chem.rdchem.Mol'>, typing.Tuple[rdkit.Chem.rdchem.Mol, papyrus_structure_pipeline.standardizer.SaltStrippingResult]]
[<class 'bytes'>, typing.List[bytes]]
[typing.List[charset_normalizer.models.CharsetMatch], <class 'NoneType'>]
[<class 'int'>, <class 'str'>]
[ForwardRef('CharsetMatch'), <class 'NoneType'>]
[<class 'bytes'>, <class 'bytearray'>]
[typing.List[str], <class 'NoneType'>]
[<class 'str'>, <class 'bytes'>, <class 'os.PathLike'>]
[<class 'os.PathLike'>, <class 'str'>, <class 'typing.BinaryIO'>, <class 'bytes'>]
[<class 'str'>, <class 'float'>]
[<class 'str'>, <class 'float'>, <class 'NoneType'>]
[<class 'bytes'>, <class 'str'>]
[<class 'str'>, <class 'bytes'>, <class 

2024-04-21 19:16:03,996	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


[<class 'str'>, <class 'logging.Logger'>]
[ForwardRef('RepeatedScalarFieldContainer[_T]'), typing.Iterable[~_T]]
[<class 'int'>, <class 'NoneType'>]
[<class 'int'>, <class 'slice'>]
[ForwardRef('RepeatedCompositeFieldContainer[_T]'), typing.Iterable[~_T]]
[~_V, <class 'NoneType'>]
[~_V, ~_T]
[<class 'dict'>, <class 'NoneType'>]
[<class 'bool'>, <class 'NoneType'>]
[typing.IO[~AnyStr], <class 'NoneType'>]
[typing.List[int], <class 'NoneType'>]
[typing.Callable[[], NoneType], <class 'NoneType'>]
[ForwardRef('_BarManager'), <class 'NoneType'>]
[typing.Iterable, <class 'NoneType'>]
[typing.List[str], <class 'NoneType'>]
[typing.Dict[str, typing.Any], ForwardRef('RuntimeEnv')]
[typing.Dict[str, typing.Any], ForwardRef('RuntimeEnv'), <class 'NoneType'>]
[ForwardRef('grpc.ChannelCredentials'), <class 'NoneType'>]
[typing.Dict[str, typing.Any], <class 'NoneType'>]
[<class 'float'>, <class 'NoneType'>]
[typing.Sequence[typing.Tuple[str, typing.Any]], <class 'NoneType'>]
[<class 'str'>, <class '

In [11]:
#Download the papyrus database
download_papyrus(only_pp=False)

Latest version: 05.6
Number of files to be downloaded: 10
Total size: 8.18GB


In [32]:
#Define Bcl-2 as target to filter for
bcl_ac = ['P10415','A0A1L4AQQ4','A0A1L4AQQ5','A0A1L4AQQ8','A0A1L4AQR0', 'A0A1L4AQR6','A0A7I2V3S7','A0A7I2V5Q7']

#Define Mcl-2 as target to filter for
mcl_ac = ['Q07820','C8YZ26','A0A7I2V2W2','A0A089WT64']

Bc-2 data

In [ ]:
data = read_papyrus(plusplus=True,chunksize=1_000_000)

In [37]:
#Read the protein data
protein_data = read_protein_set()

In [38]:
#Keep only high quality data
filter1=keep_quality(data,'High')

In [39]:
#Create the dataset
filter_bcl=keep_accession(filter1, bcl_ac)


In [40]:
bcl = consume_chunks(filter_bcl)

0it [00:00, ?it/s]

In [41]:
bcl.to_csv('bcl/bcl2_raw.csv',index=True)

Mcl-1 data

In [50]:
data = read_papyrus(plusplus=True,chunksize=1_000_000)

In [51]:
#Read the protein data
protein_data = read_protein_set()

In [52]:
filter1=keep_quality(data,'High')

In [53]:
filter_mcl=keep_accession(filter1, mcl_ac)

In [54]:
mcl = consume_chunks(filter_mcl)

0it [00:00, ?it/s]

In [55]:
#Save the raw query result
mcl.to_csv('mcl/mcl1_raw.csv',index=True)

Clean

In [12]:
df_bcl = bcl[['SMILES','InChIKey','connectivity','pchembl_value_Mean']]
df_mcl = mcl[['SMILES','InChIKey','connectivity','pchembl_value_Mean']]

In [14]:
def compare_values(row):
    if row['pchembl_value_Mean'] < 6.5:
        return 'Non-inhibitor'
    elif row['pchembl_value_Mean'] >= 6.5:
        return 'Inhibitor'
    else:
        return '??'

df_bcl['Class'] = df_bcl.apply(compare_values, axis=1)
df_mcl['Class'] = df_mcl.apply(compare_values, axis=1)


,SMILES,InChIKey,connectivity,pchembl_value_Mean,Class
2738,Cc1cc(=O)c2c(o1)c(CC(C)C)c(O)c(O)c2O,ADNKLAAIRXDEOP-UHFFFAOYSA-N,ADNKLAAIRXDEOP,6.14,Non-inhibitor
3027,N#CC1=C(O)c2cccc3c(Sc4ccc(Br)cc4)ccc(c23)C1=NC...,AEAIKBHAZFBIFT-UHFFFAOYSA-N,AEAIKBHAZFBIFT,5.99,Non-inhibitor
4466,COC1(CC(C)C)CCN(c2ccc(C(=O)NS(=O)(=O)c3ccc(NC(...,AGGPTAKEWABCFB-UHFFFAOYSA-N,AGGPTAKEWABCFB,7.25,Inhibitor
5302,O=C1c2c(cccc2)C(=O)c2c1cc(O)c(O)c2O,AHKDJQYHVWSRLT-UHFFFAOYSA-N,AHKDJQYHVWSRLT,5.88,Non-inhibitor
5921,CC1=CC(C=C(Cl)c2ccc(C(=O)O)cc2)C2(C(=O)O)C=CC(...,AIPCKDBTGKGQEY-UHFFFAOYSA-N,AIPCKDBTGKGQEY,6.43,Non-inhibitor


In [15]:
#Check for duplicates 

df = df_bcl #Change the dataset here, to check for duplicates

print(len(df))
print(df['connectivity'].nunique())
print(df['connectivity'].value_counts())

#Check for contradicting duplicates
unique_counts_log = df.groupby('connectivity')['Class'].nunique()
duplicates_diff_class_log = unique_counts_log[unique_counts_log > 1].index

print(duplicates_diff_class_log)
print(f'Contradicting duplicates: {len(duplicates_diff_class_log)}')

df = df.reset_index(drop=True)

#Save final datasets without duplicates: Change filenames!
df.to_csv('bcl/bcl_molecules.csv', index=True)

428
428
ADNKLAAIRXDEOP    1
SNXZLJPTQJSGBF    1
SMQYDTWNUNHOFH    1
SMMOVTLHWLZLTP    1
SLACVTBRGGXEOK    1
                 ..
JIWLBMMRHHWXRO    1
JEOIJXVKYVIEPU    1
JECDGNOBGBDIPY    1
JBFLNNJFYIKVDD    1
ZZQJFMAANHFPSS    1
Name: connectivity, Length: 428, dtype: int64
Index([], dtype='object', name='connectivity')
Contradicting duplicates: 0
